# Video Tokenization Using [NVIDIA Cosmos Tokenizer](https://github.com/NVIDIA/Cosmos-Tokenizer) | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nvidia/Cosmos-Tokenizer/blob/main/notebook/Video_Tokenization.ipynb)

The Jupyter Notebook example utilizes the **Cosmos-Tokenizer** pretrained models, which include Continuous Video (CV) tokenizers that transform videos into continuous spatio-temporal latents and Discrete Video (DI) tokenizers that transform videos into discrete tokens. Both CV and DV tokenizers are available with compression rates of (`TxHxW` format) 4x8x8 and 8x8x8, and 8x16x16. For instance, **CV4x8x8** effectively downsizes the number of frames by a factor of 4 and both height and width by a factor of 8.

Within the notebook, the `VideoTokenizer` class from the `cosmos_tokenizer.video_lib` module is employed to manage the encoder and decoder components of this model. The encoder compresses the input video into a condensed latent representation or discrete integers, while the decoder reconstructs the video from this latent representation or discrete integers.

This instance of the Cosmos Tokenizer demonstrates its autoencoding capability: compressing a video into a smaller latent space and subsequently reconstructing it to its original form. This showcases the efficiency of video tokenization for tasks involving significant spatial compression during video reconstruction, a highly desirable feature for generative modeling.


This tutorial follows a simple, step-by-step approach, making it easy to understand and adapt.

## Step 1: Clone the Cosmos Tokenizer Repository

In [1]:
!git clone https://github.com/NVIDIA/Cosmos-Tokenizer.git

Cloning into 'Cosmos-Tokenizer'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 197 (delta 37), reused 33 (delta 17), pack-reused 121 (from 1)
Receiving objects: 100% (197/197), 11.37 MiB | 16.06 MiB/s, done.
Resolving deltas: 100% (99/99), done.


## Step 2: Install **Cosmos-Tokenizer**
Before proceeding, ensure you have the **Cosmos Tokenizer** installed. If you cloned the repository in Step 1, use the following command to install it in editable mode:

In [2]:
# Step 2: # Install Cosmos-Tokenizer and its Python dependencies.
import os
if os.path.exists("Cosmos-Tokenizer"):
    os.chdir("Cosmos-Tokenizer")
    !apt-get update
    !apt-get install -y git-lfs
    !git lfs pull
    %pip install -e .
else:
    print('Cosmos-Tokenizer is already installed.')

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,159 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/u

## Step 3: Set Up Hugging Face API Token and Download Pretrained Models

In this step, you'll configure the Hugging Face API token and download the pretrained model weights required for the **Cosmos Tokenizer**.

1. **Ensure You Have a Hugging Face Account**  
   If you do not already have a Hugging Face account, follow these steps to create one and generate an API token:
   - Go to the [Hugging Face website](https://huggingface.co/) and sign up for a free account.
   - After logging in, navigate to your [Settings → Access Tokens](https://huggingface.co/settings/tokens).
   - Click on "New Token" to generate an API token with the required permissions.

2. **Set the Hugging Face Token**  
   Check if the Hugging Face token is already set in the environment variables. If not, you will be prompted to enter it manually. The token is essential to authenticate and access the Hugging Face models.



In [7]:
# Check if the token is already set
if "HUGGINGFACE_TOKEN" not in os.environ:
    os.environ["HUGGINGFACE_TOKEN"] = input("Please enter your Hugging Face API token: ")
!git config --global credential.helper store

In [9]:
from huggingface_hub import login, snapshot_download
import os
HUGGINGFACE_TOKEN = os.environ.get("HUGGINGFACE_TOKEN")
login(token=HUGGINGFACE_TOKEN, add_to_git_credential=True)
model_names = [
        "Cosmos-0.1-Tokenizer-CV4x8x8",
        "Cosmos-0.1-Tokenizer-CV8x16x16",
        "Cosmos-0.1-Tokenizer-DV4x8x8",
        "Cosmos-0.1-Tokenizer-DV8x8x8",
        "Cosmos-0.1-Tokenizer-DV8x16x16",
        "Cosmos-1.0-Tokenizer-CV8x8x8",
        "Cosmos-1.0-Tokenizer-DV8x16x16",
]
for model_name in model_names:
    hf_repo = "nvidia/" + model_name
    local_dir = "pretrained_ckpts/" + model_name
    os.makedirs(local_dir, exist_ok=True)
    print(f"downloading {model_name}...")
    snapshot_download(repo_id=hf_repo, local_dir=local_dir)

downloading Cosmos-0.1-Tokenizer-CV4x8x8...


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

downloading Cosmos-0.1-Tokenizer-CV8x16x16...


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

downloading Cosmos-0.1-Tokenizer-DV4x8x8...


...

downloading Cosmos-0.1-Tokenizer-DV8x16x16...


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

downloading Cosmos-1.0-Tokenizer-CV8x8x8...


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

GatedRepoError: 403 Client Error. (Request ID: Root=1-692fad58-5d9afe92465281ec43bc7e40;433bdbee-c9c2-4d44-a055-6997e911e85d)

Cannot access gated repo for url https://huggingface.co/nvidia/Cosmos-1.0-Tokenizer-CV8x8x8/resolve/5110dbb3c94a9eb6648eabbbccc19c76180fe426/.gitattributes.
Access to model nvidia/Cosmos-1.0-Tokenizer-CV8x8x8 is restricted and you are not in the authorized list. Visit https://huggingface.co/nvidia/Cosmos-1.0-Tokenizer-CV8x8x8 to ask for access.

## Step 4: Use Cosmos Tokenizer for Video Reconstruction



In [6]:
# @title In this step, load the required checkpoints, and perform video reconstruction. {"run":"auto"}
import cv2
import numpy as np
import torch

import importlib
import cosmos_tokenizer.video_lib
import mediapy as media

importlib.reload(cosmos_tokenizer.video_lib)
from cosmos_tokenizer.video_lib import CausalVideoTokenizer

# 1) Specify the model name, and the paths to the encoder/decoder checkpoints.
model_name = 'Cosmos-1.0-Tokenizer-CV8x8x8' # @param ["Cosmos-0.1-Tokenizer-CV4x8x8", "Cosmos-0.1-Tokenizer-CV8x8x8", "Cosmos-0.1-Tokenizer-CV8x16x16", "Cosmos-0.1-Tokenizer-DV4x8x8", "Cosmos-0.1-Tokenizer-DV8x8x8", "Cosmos-0.1-Tokenizer-DV8x16x16", "Cosmos-1.0-Tokenizer-CV8x8x8", "Cosmos-1.0-Tokenizer-DV8x16x16"]
temporal_window = 49 # @param {type:"slider", min:1, max:121, step:8}

encoder_ckpt = f"pretrained_ckpts/{model_name}/encoder.jit"
decoder_ckpt = f"pretrained_ckpts/{model_name}/decoder.jit"

# 2) Load or provide the video filename you want to tokenize & reconstruct.
input_filepath = "test_data/video.mp4"

# 3) Read the video from disk (shape = T x H x W x 3 in BGR).
input_video = media.read_video(input_filepath)[..., :3]
assert input_video.ndim == 4 and input_video.shape[-1] == 3, "Frames must have shape T x H x W x 3"

# 4) Expand dimensions to B x Tx H x W x C, since the CausalVideoTokenizer expects a batch dimension
#    in the input. (Batch size = 1 in this example.)
batched_input_video = np.expand_dims(input_video, axis=0)

# 5) Create the CausalVideoTokenizer instance with the encoder & decoder.
#    - device="cuda" uses the GPU
#    - dtype="bfloat16" expects Ampere or newer GPU (A100, RTX 30xx, etc.)
tokenizer = CausalVideoTokenizer(
    checkpoint_enc=encoder_ckpt,
    checkpoint_dec=decoder_ckpt,
    device="cuda",
    dtype="bfloat16",
)

# 6) Use the tokenizer to autoencode (encode & decode) the video.
#    The output is a NumPy array with shape = B x T x H x W x C, range [0..255].
batched_output_video = tokenizer(batched_input_video,
                                 temporal_window=temporal_window)

# 7) Extract the single video from the batch (index 0).
output_video = batched_output_video[0]

# 9) Save the reconstructed video to disk.
input_dir, input_filename = os.path.split(input_filepath)
filename, ext = os.path.splitext(input_filename)
output_filepath = f"{input_dir}/{filename}_{model_name.split('-')[-1]}{ext}"
media.write_video(output_filepath, output_video)
print("Input video read from:\t", f"{os.getcwd()}/{input_filepath}")
print("Reconstruction saved:\t", f"{os.getcwd()}/{output_filepath}")

# 10) Visualization of the input video (left) and the reconstruction (right).
media.show_videos([input_video, output_video], ["Input Video", "Reconstructed Video"], height=720)

ValueError: The provided filename pretrained_ckpts/Cosmos-1.0-Tokenizer-CV8x8x8/encoder.jit does not exist